In [17]:
import numpy as np
import pandas as pd
from time import time,sleep
from random import randint as r
import random
import matplotlib.pyplot as plt
import seaborn as sns
import time

In [18]:
train = pd.read_csv('train.csv', index_col=0)
test = pd.read_csv('test.csv', index_col=0)
centroids = pd.read_csv('centroids.csv')
init_lims = 1/np.sqrt(centroids.shape[0])*10000

#### Initialise Weights

In [19]:
w = np.random.randint(-int(init_lims), int(init_lims), (2,7))/10000

#### Create Training Datasets

In [20]:
X_train = train[['x1', 'x2']]
Y_train = train['Y']
mu = centroids[['mu1', 'mu2']]
sigma = centroids['sigma']
eta = 0.01

In [21]:
Y_train = Y_train.reset_index()
Y_train = Y_train.drop('index', axis=1)
Y_train['pos_1'] = 0
Y_train['neg_1'] = 0

In [22]:
for i in range(len(Y_train)):
    if Y_train['Y'][i] == 1:
        Y_train['neg_1'][i] = 0
        Y_train['pos_1'][i] = 1
    if Y_train['Y'][i] == -1:
        Y_train['neg_1'][i] = 1
        Y_train['pos_1'][i] = 0
Y_train = Y_train[['Y', 'pos_1','neg_1']]
o = Y_train[['pos_1', 'neg_1']]

#### Calculate h (hidden layer output)

In [23]:
sub_X_mui = np.array(X_train) - np.array(mu.loc[0])
working = pd.DataFrame(sub_X_mui)
h_df = pd.DataFrame() 
for i in range(centroids.shape[0]):
    column_name = 'h'+ str(i+1)
    sub_X_mui = np.array(X_train) - np.array(mu.loc[i])
    working = pd.DataFrame(sub_X_mui)
    h_df[column_name] = working[0]**2 + working[1]**2
    h_df[column_name] = h_df[column_name]/(2*sigma[i]**2)
    h_df[column_name] = np.exp(-h_df[column_name])
h_df['b'] = -1
h_df_trans = h_df.T

In [24]:
convergence_flag = False
x_loc=0
iteration = 0
while (convergence_flag == False):
    if x_loc == 500: x_loc = 0
        
    # Calculate Outputs    
    output_pos_1 = np.dot(h_df.T[x_loc],w[0])
    output_neg_1 = np.dot(h_df.T[x_loc],w[1])
    
    # Calculate Weight Delta
    weight_delta_1 = (o['pos_1'][x_loc]-output_pos_1)*h_df.T[x_loc]*eta
    weight_delta_2 = (o['neg_1'][x_loc]-output_neg_1)*h_df.T[x_loc]*eta
    
    # Update Weights
    weight_delta = np.array([weight_delta_1, weight_delta_2])
    w = w + weight_delta
    
    x_loc = x_loc + 1
    iteration = iteration + 1 

    if weight_delta.max() < 0.0001:
        convergence_flag=True
        print("converged")
        print("iteration = ", iteration)
    if iteration == 5000:
        convergence_flag=True
        print("iteration = 5000")

converged
iteration =  1893


In [29]:
w

array([[ 0.40596071, -0.44019586, -0.66804018,  0.53514567,  0.40940273,
        -0.49273231, -0.53468528],
       [-0.2614817 ,  0.33265251,  0.59699873, -0.50045748, -0.56002621,
         0.46945188, -0.48374142]])

In [25]:
pd.DataFrame(w).to_csv('RBFNN_weights_chris.csv')

PermissionError: [Errno 13] Permission denied: 'RBFNN_weights_chris.csv'

In [ ]:
w

In [26]:
X_test = test[['x1','x2']]
sub_X_mui = np.array(X_test) - np.array(mu.loc[0])
working = pd.DataFrame(sub_X_mui)
h_df = pd.DataFrame() 
for i in range(centroids.shape[0]):
    column_name = 'h'+ str(i+1)
    sub_X_mui = np.array(X_test) - np.array(mu.loc[i])
    working = pd.DataFrame(sub_X_mui)
    h_df[column_name] = working[0]**2 + working[1]**2
    h_df[column_name] = h_df[column_name]/(2*sigma[i]**2)
    h_df[column_name] = np.exp(-h_df[column_name])
h_df['b'] = -1
h_df_trans = h_df.T

In [27]:
Y_pred = []
Y_test = test['Y.test']
for i in range(Y_test.shape[0]):
    output_pos_1 = np.dot(h_df.T[i],w[0])
    output_neg_1 = np.dot(h_df.T[i],w[1])
    if output_pos_1 > output_neg_1: 
        Y_pred.append(1)
    else:
        Y_pred.append(-1)

In [28]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_test, Y_pred)

array([[48,  5],
       [ 3, 44]], dtype=int64)

In [14]:
sub_X_mui = np.array(X_train) - np.array(mu.loc[0])
working = pd.DataFrame(sub_X_mui)
h_df = pd.DataFrame() 
for i in range(centroids.shape[0]):
    column_name = 'h'+ str(i+1)
    sub_X_mui = np.array(X_train) - np.array(mu.loc[i])
    working = pd.DataFrame(sub_X_mui)
    h_df[column_name] = working[0]**2 + working[1]**2
    h_df[column_name] = h_df[column_name]/(2*sigma[i]**2)
    h_df[column_name] = np.exp(-h_df[column_name])
h_df['b'] = -1
h_df_trans = h_df.T

In [15]:
Y_train_pred = []
Y_train = train['Y']
for i in range(Y_train.shape[0]):
    output_pos_1 = np.dot(h_df.T[i],w[0])
    output_neg_1 = np.dot(h_df.T[i],w[1])
    if output_pos_1 > output_neg_1: 
        Y_train_pred.append(1)
    else:
        Y_train_pred.append(-1)

In [16]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_train, Y_train_pred)

array([[222,  17],
       [ 27, 234]], dtype=int64)

In [38]:
Y_test.value_counts()

-1    53
 1    47
Name: Y.test, dtype: int64

In [ ]:
np.set_printoptions(precision=2)
# Plot non-normalized confusion matrix
titles_options = [("Confusion matrix, without normalization", None),
                  ("Normalized confusion matrix", 'true')]
for title, normalize in titles_options:
    disp = plot_confusion_matrix(rf_OT, x_test_0_1, y_test_1_0,
                                 display_labels='OTIF',
                                 cmap=plt.cm.Blues,
                                 normalize=normalize)
    disp.ax_.set_title(title)
    print(title)
    print(disp.confusion_matrix)
    plt.savefig('../visualisations/RF_confusion Matrix_'+title+'.png',bbox_inches = 'tight');


In [ ]:
# sub_X_mui = np.array(X_train) - np.array(mu.loc[0])
# working = pd.DataFrame(sub_X_mui)
# working['h'] = working[0]**2 + working[1]**2
# working['h'] = working['h']/(2*sigma[0]**2)
# working['h'] = np.exp(-working['h'])

#### Calculate Outputs

In [ ]:
output_pos_1 = np.dot(h_df.T[0],w[0])
output_neg_1 = np.dot(h_df.T[0],w[1])

#### Calculate Weight Delta

In [ ]:
weight_delta_1 = (o['pos_1'][0]-output_pos_1)*h_df.T[0]*eta
weight_delta_2 = (o['neg_1'][0]-output_neg_1)*h_df.T[0]*eta

#### Update Weights

In [ ]:
weight_delta_1

In [ ]:
weight_delta = np.array([weight_delta_1, weight_delta_2])

In [ ]:
weight_delta

In [ ]:
w

In [ ]:
w = w + weight_delta

In [ ]:
weight_delta.max()

In [ ]:
np.dot(h_df.T[0],w[0])

In [ ]:
np.dot(h_df.T[0],w[1])

In [ ]:
h_df.T

In [ ]:
o['pos_1'][0]-np.dot(h_df.T[0],w[0])*h_df.T[0]*eta

In [ ]:
np.dot(h_df.T[0],w[1])

In [ ]:
sub_X_mui = np.array(X_train) - np.array(mu.loc[0])
X_mu_square = np.square(sub)

In [ ]:
X_mu_square

In [ ]:
def rbf(x, mu, sigma):
    
    ans = np.exp(x-mu)

In [ ]:
train

In [ ]:
ax = sns.scatterplot(x="x1", y="x2", data=train, hue="Y")

In [ ]:
train['Y'].value_counts()